In [1]:
import numpy as np
import pandas as pd

import torch
from torch import nn
from torch.nn import functional as F

import torchvision
from torch import optim

from matplotlib import pyplot as plt
from IPython.display import clear_output
from collections import OrderedDict

In [2]:
import os
from tqdm import tqdm
from sklearn.model_selection import train_test_split

In [3]:
def rgba2rgb( rgba, background=(255,255,255) ):
    rgba = np.array(rgba)
    row, col, ch = rgba.shape

    if ch == 3:
        return rgba

    assert ch == 4, 'RGBA image has 4 channels.'

    rgb = np.zeros( (row, col, 3), dtype='float32' )
    r, g, b, a = rgba[:,:,0], rgba[:,:,1], rgba[:,:,2], rgba[:,:,3]

    a = np.asarray( a, dtype='float32' ) / 255.0

    R, G, B = background

    rgb[:,:,0] = r * a + (1.0 - a) * R
    rgb[:,:,1] = g * a + (1.0 - a) * G
    rgb[:,:,2] = b * a + (1.0 - a) * B

    return np.asarray( rgb, dtype='uint8' )

In [4]:
import numpy
from PIL import Image

def get_image(image_path):
    """Get a numpy array of an image so that one can access values[x][y]."""
    image = Image.open(image_path, "r")
    image = image.resize((300, 200)) 
    image = rgba2rgb(image)
    #image = image.convert('RGB')
    img_array = np.array(image)
    return img_array

In [5]:
def return_files(path):
    dir_list = os.listdir(path)
    files = []
    for file in tqdm(dir_list):
        files.append(path + '\\' + file)
    return files

In [7]:
fl2 = return_files(r"calc_img3 2\calc_img3\D_\Code\Petrukhin\fee\client_open\client_project")

100%|███████████████████████████████████████████████████████████████████████| 18098/18098 [00:00<00:00, 1291686.04it/s]


In [8]:
fl3 = return_files(r"calc_img3 3\calc_img3\D_\Code\Petrukhin\fee\client_open\client_project")

100%|█████████████████████████████████████████████████████████████████████| 139518/139518 [00:00<00:00, 2295214.12it/s]


In [9]:
fl4 = return_files(r'calc_ing4\calc_ing4\D_\Code\Petrukhin\fee\client_open\client_project')

100%|█████████████████████████████████████████████████████████████████████| 154874/154874 [00:00<00:00, 2187189.22it/s]


In [10]:
files = fl1 + fl2 + fl3 + fl4

In [11]:
def return_image_and_label(file):
    splits = file.split('_')
    image = get_image(file)
    percent = float(splits[-1][:-4])
    
    label = 0
    if percent > 3.:
        label = 1
        
    return image, label

In [12]:
def calc_dist(files):
    d = {0: 0, 1: 0, 2: 0}
    for file in tqdm(files):
        splits = file.split('_')
        percent = float(splits[-1][:-4])
        label = 0
        if percent >= 3.:
            label = 1
        yield label

In [13]:
y = list(calc_dist(files))

100%|██████████████████████████████████████████████████████████████████████| 315761/315761 [00:00<00:00, 958093.55it/s]


In [14]:

import numpy as np 
from sklearn.utils.class_weight import compute_class_weight 
class_weight = compute_class_weight(class_weight="balanced", classes=np.unique(y), y=y)

In [16]:
import torch.nn.functional as F 
 
class DropBlock(nn.Module): 
    def __init__(self, block_size: int, p: float = 0.5): 
        super().__init__() 
        self.block_size = block_size 
        self.p = p 
 
 
    def calculate_gamma(self, x) -> float: 
        """Compute gamma, eq (1) in the paper 
        Args: 
            x (Tensor): Input tensor 
        Returns: 
            Tensor: gamma 
        """ 
         
        invalid = (1 - self.p) / (self.block_size ** 2) 
        valid = (x.shape[-1] ** 2) / ((x.shape[-1] - self.block_size + 1) ** 2) 
        return invalid * valid 
 
 
 
    def forward(self, x): 
        if self.training: 
            gamma = self.calculate_gamma(x) 
            mask = torch.bernoulli(torch.ones_like(x) * gamma) 
            mask_block = 1 - F.max_pool2d( 
                mask, 
                kernel_size=(self.block_size, self.block_size), 
                stride=(1, 1), 
                padding=(self.block_size // 2, self.block_size // 2), 
            ) 
            x = mask_block * x * (mask_block.numel() / mask_block.sum()) 
        return x

In [17]:
import torch
import torch.nn as nn
import torch.nn.functional as F

def drop_block_2d(
        x,
        drop_prob: float = 0.1,
        block_size: int = 7,
        gamma_scale: float = 1.0,
        with_noise: bool = False,
        inplace: bool = False,
        batchwise: bool = False
):
    """ DropBlock. See https://arxiv.org/pdf/1810.12890.pdf

    DropBlock with an experimental gaussian noise option. This layer has been tested on a few training
    runs with success, but needs further validation and possibly optimization for lower runtime impact.
    """
    B, C, H, W = x.shape
    total_size = W * H
    clipped_block_size = min(block_size, min(W, H))
    # seed_drop_rate, the gamma parameter
    gamma = gamma_scale * drop_prob * total_size / clipped_block_size ** 2 / (
            (W - block_size + 1) * (H - block_size + 1))

    # Forces the block to be inside the feature map.
    w_i, h_i = ndgrid(torch.arange(W, device=x.device), torch.arange(H, device=x.device))
    valid_block = ((w_i >= clipped_block_size // 2) & (w_i < W - (clipped_block_size - 1) // 2)) & \
                  ((h_i >= clipped_block_size // 2) & (h_i < H - (clipped_block_size - 1) // 2))
    valid_block = torch.reshape(valid_block, (1, 1, H, W)).to(dtype=x.dtype)

    if batchwise:
        # one mask for whole batch, quite a bit faster
        uniform_noise = torch.rand((1, C, H, W), dtype=x.dtype, device=x.device)
    else:
        uniform_noise = torch.rand_like(x)
    block_mask = ((2 - gamma - valid_block + uniform_noise) >= 1).to(dtype=x.dtype)
    block_mask = -F.max_pool2d(
        -block_mask,
        kernel_size=clipped_block_size,  # block_size,
        stride=1,
        padding=clipped_block_size // 2)

    if with_noise:
        normal_noise = torch.randn((1, C, H, W), dtype=x.dtype, device=x.device) if batchwise else torch.randn_like(x)
        if inplace:
            x.mul_(block_mask).add_(normal_noise * (1 - block_mask))
        else:
            x = x * block_mask + normal_noise * (1 - block_mask)
    else:
        normalize_scale = (block_mask.numel() / block_mask.to(dtype=torch.float32).sum().add(1e-7)).to(x.dtype)
        if inplace:
            x.mul_(block_mask * normalize_scale)
        else:
            x = x * block_mask * normalize_scale
    return x

def drop_block_fast_2d_old(
        x: torch.Tensor,
        drop_prob: float = 0.1,
        block_size: int = 7,
        gamma_scale: float = 1.0,
        with_noise: bool = False,
        inplace: bool = False,
):
    """ DropBlock. See https://arxiv.org/pdf/1810.12890.pdf

    DropBlock with an experimental gaussian noise option. Simplied from above without concern for valid
    block mask at edges.
    """
    B, C, H, W = x.shape
    total_size = W * H
    clipped_block_size = min(block_size, min(W, H))
    gamma = gamma_scale * drop_prob * total_size / clipped_block_size ** 2 / (
            (W - block_size + 1) * (H - block_size + 1))

    block_mask = torch.empty_like(x).bernoulli_(gamma)
    block_mask = F.max_pool2d(
        block_mask.to(x.dtype), kernel_size=clipped_block_size, stride=1, padding=clipped_block_size // 2)
    print(block_mask)
    if with_noise:
        normal_noise = torch.empty_like(x).normal_()
        if inplace:
            x.mul_(1. - block_mask).add_(normal_noise * block_mask)
        else:
            x = x * (1. - block_mask) + normal_noise * block_mask
    else:
        block_mask = 1 - block_mask
        normalize_scale = (block_mask.numel() / block_mask.to(dtype=torch.float32).sum().add(1e-6)).to(dtype=x.dtype)
        if inplace:
            x.mul_(block_mask * normalize_scale)
        else:
            x = x * block_mask# * normalize_scale
    return x

def drop_block_fast_2d(
        x: torch.Tensor,
        drop_prob: float = 0.1,
        block_size: int = 7,
        gamma_scale: float = 1.0,
        with_noise: bool = False,
        inplace: bool = False,
):
    """ DropBlock. See https://arxiv.org/pdf/1810.12890.pdf

    DropBlock with an experimental gaussian noise option. Simplied from above without concern for valid
    block mask at edges.
    """
    B, C, H, W = x.shape
    total_size = W * H
    clipped_block_size = min(block_size, min(W, H))
    gamma = gamma_scale * drop_prob * total_size / clipped_block_size ** 2 / (
            (W - block_size + 1) * (H - block_size + 1))

    block_mask = torch.empty_like(x).bernoulli_(gamma)
    block_mask = F.max_pool3d(
        block_mask.to(x.dtype), 
        kernel_size=clipped_block_size, stride=1, 
        padding=clipped_block_size // 2
    )
    x = x * (1 - block_mask) + block_mask
    return x


class DropBlock2d(nn.Module):
    """ DropBlock. See https://arxiv.org/pdf/1810.12890.pdf
    """
    def __init__(
            self,
            drop_prob: float = 0.1,
            block_size: int = 7,
            gamma_scale: float = 1.0,
            with_noise: bool = False,
            inplace: bool = False,
            batchwise: bool = False,
            fast: bool = True):
        super(DropBlock2d, self).__init__()
        self.drop_prob = drop_prob
        self.gamma_scale = gamma_scale
        self.block_size = block_size
        self.with_noise = with_noise
        self.inplace = inplace
        self.batchwise = batchwise
        self.fast = fast  # FIXME finish comparisons of fast vs not

    def forward(self, x):
        if not self.training or not self.drop_prob:
            return x
        if self.fast:
            return drop_block_fast_2d(
                x, self.drop_prob, self.block_size, self.gamma_scale, self.with_noise, self.inplace)
        else:
            return drop_block_2d(
                x, self.drop_prob, self.block_size, self.gamma_scale, self.with_noise, self.inplace, self.batchwise)

In [18]:
from torch.utils.data import Dataset
class TrainDataset(Dataset):
    def __init__(self, files, transform=None, target_transform=None):
        super().__init__()
        self.files = files
        self.transform = transform
        self.target_transform = target_transform
        #self.drop_block = DropBlock2d(drop_prob=0.1,block_size=19)#DropBlock(block_size=19, p=0.8) 

    def __len__(self):
        return len(self.files)

    def __getitem__(self, index):
        image, label = return_image_and_label(self.files[index])
        
        image = torch.from_numpy(image)
        image = image.permute(2, 1, 0) / 255
        
        #image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
        #image = self.drop_block(image)
        #image = image.reshape(( image.shape[1], image.shape[2], image.shape[3]))
        
        label = torch.LongTensor([label])
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label

In [19]:
class TestDataset(Dataset):
    def __init__(self, files, transform=None):
        super().__init__()
        self.files = files
        self.transform = transform


    def __len__(self):
        return len(self.img_array)

    def __getitem__(self, idx):
        image, _ = return_image_and_label(self.files[index])
        
        image = torch.from_numpy(image)
        image = image.permute(2, 1, 0) / 255
        if self.transform:
            image = self.transform(image)

        return image

In [20]:
from sklearn.model_selection import train_test_split

In [21]:
import torchvision
from torchvision.io import read_image
from torchvision import transforms

In [22]:
# Задаём трансформеры
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize(224, antialias=True),
        #transforms.RandomRotation(degrees=(-10, 10)),
    ]),
    'test': transforms.Compose([
        transforms.Resize(224, antialias=True),
        #transforms.CenterCrop(224),
    ]),
}

In [23]:
train_data, test_data = train_test_split(files)

In [24]:
train_dataset = TrainDataset(train_data)
test_dataset = TrainDataset(test_data)

In [25]:
train_dataset[0][0].shape

torch.Size([3, 300, 200])

In [26]:
train_dataset[0][0].shape

torch.Size([3, 300, 200])

In [29]:
from torch.utils.data import Dataset, DataLoader
train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=8, shuffle=False)

In [30]:
def train_step(model, dataloader, loss_fn, optimizer, scheduler=None):
    model.train()

    train_loss, train_acc = 0, 0

    for batch in tqdm(dataloader):
        inputs, labels = batch
        #labels = labels.long()
        inputs, labels = inputs.cuda(), labels.reshape(-1).cuda()

        outputs = model(inputs)
        
        #print('outputs: ', outputs.shape)
        #print('labels: ', labels.shape)
        #print('labels: ', labels)

        loss = loss_fn(outputs, labels)
        train_loss += loss.item() 
        optimizer.zero_grad()

        loss.backward()
         
        optimizer.step()

        train_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        train_acc += (predicted == labels).sum().item()
    if scheduler:
        scheduler.step()
    epoch_train_loss = train_loss / float(len(train_data))
    epoch_train_accuracy = train_acc / float(len(train_data))

    return epoch_train_loss, epoch_train_accuracy


In [31]:
def val_step(model, dataloader, loss_fn, scheduler):
    model.eval()
    
    val_loss, val_acc = 0, 0

    with torch.inference_mode():
        for batch in tqdm(dataloader):
            inputs, labels = batch
            #labels = labels.long()
            inputs, labels = inputs.cuda(), labels.reshape(-1).cuda()

            outputs = model(inputs)

            loss = loss_fn(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            val_acc += (predicted == labels).sum().item()

    epoch_val_loss = val_loss / float(len(test_data))
    epoch_val_acc = val_acc / float(len(test_data))
    if scheduler:
        scheduler.step(epoch_val_loss)

    return epoch_val_loss, epoch_val_acc

In [32]:
def train(model, train_dataloader, val_dataloader, loss_fn, optimizer, epochs, scheduler_train=None, scheduler_val=None, writer=None):
    results = {
        'train_loss': [],
        'train_acc': [],
        'val_loss': [],
        'val_acc': []
    }

    for epoch in range(10000):
        train_loss, train_acc = train_step(model=model,
                                           dataloader=train_dataloader,
                                           loss_fn=loss_fn,
                                           optimizer=optimizer,
                                           scheduler=scheduler_train)
        val_loss, val_acc = val_step(model=model,
                                      dataloader=val_dataloader,
                                      loss_fn=loss_fn,
                                      scheduler=scheduler_val)

        print(f'Train loss: {train_loss:.4f}, train acc : {train_acc:.4f}. Val loss: {val_loss:.4f}, val acc : {val_acc:.4f}.')

        #if epoch % 10 == 0:
        #    torch.save(model.state_dict(), f'epoch{epoch}_{model.name}.pth')

        results['train_loss'].append(train_loss)
        results['train_acc'].append(train_acc)
        results['val_loss'].append(val_loss)
        results['val_acc'].append(val_acc)

        if val_loss < min(results['val_loss']):
            torch.save(model.state_dict(), f'best_loss_{model.name}.pth') # where???

    writer.close()
    return results

In [33]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride = 1, downsample = None):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Sequential(
                        nn.Conv2d(in_channels, out_channels, kernel_size = 3, stride = stride, padding = 1),
                        nn.BatchNorm2d(out_channels),
                        nn.ReLU())
        self.conv2 = nn.Sequential(
                        nn.Conv2d(out_channels, out_channels, kernel_size = 3, stride = 1, padding = 1),
                        nn.BatchNorm2d(out_channels))
        self.downsample = downsample
        self.relu = nn.ReLU()
        self.out_channels = out_channels
        
    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.conv2(out)
        if self.downsample:
            residual = self.downsample(x)
        out += residual
        out = self.relu(out)
        return out

In [34]:
class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes = 2):
        super(ResNet, self).__init__()
        self.inplanes = 64
        self.conv1 = nn.Sequential(
                        nn.Conv2d(3, 64, kernel_size = 7, stride = 2, padding = 3),
                        nn.BatchNorm2d(64),
                        nn.ReLU())
        self.maxpool = nn.MaxPool2d(kernel_size = 3, stride = 2, padding = 1)
        self.layer0 = self._make_layer(block, 64, layers[0], stride = 1)
        self.layer1 = self._make_layer(block, 128, layers[1], stride = 2)
        self.layer2 = self._make_layer(block, 256, layers[2], stride = 2)
        self.layer3 = self._make_layer(block, 512, layers[3], stride = 2)
        self.avgpool = nn.AvgPool2d(7, stride=1)
        self.fc = nn.Linear(2048, num_classes)
        self.drp = nn.Dropout(0.2)
        self.drop_block = DropBlock2d(drop_prob=0.2,block_size=19)
        
    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes:
            
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes, kernel_size=1, stride=stride),
                nn.BatchNorm2d(planes),
            )
        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)
    
    
    def forward(self, x):
        #x = self.drop_block(x)
        x = self.conv1(x)
        x = self.maxpool(x)
        x = self.layer0(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        #print('layer3:', x.shape)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.drp(x)
        #print('SHAPE:', x.shape)
        x = self.fc(x)

        return x

In [35]:
from typing import Type, Union

In [36]:
num_classes = 10
num_epochs = 20
batch_size = 16
learning_rate = 0.01

model = ResNet(ResidualBlock, [3, 4, 6, 3]).cuda()

In [37]:
import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn
from torch import optim
from torch.optim import lr_scheduler

In [38]:
_class_weights = torch.FloatTensor(class_weight).cuda()

In [39]:
lr=1e-4

loss = nn.CrossEntropyLoss(weight=_class_weights)

optimizer = optim.AdamW(model.parameters(), lr = 1e-4)
step_scheduler = lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)
plateau_scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5, verbose=True)

In [40]:
model_results = train(model=model,
                      train_dataloader=train_dataloader,
                      val_dataloader=test_dataloader,
                      loss_fn=loss,
                      optimizer=optimizer,
                      #scheduler_val=plateau_scheduler,
                      epochs=200
                     )

100%|██████████████████████████████████████████████████████████████████████████████| 9868/9868 [05:37<00:00, 29.22it/s]


Train loss: 0.1464, train acc : 0.7359. Val loss: 0.0665, val acc : 0.7306.


100%|██████████████████████████████████████████████████████████████████████████████| 9868/9868 [05:02<00:00, 32.64it/s]


Train loss: 0.1325, train acc : 0.7432. Val loss: 0.0643, val acc : 0.8308.


100%|██████████████████████████████████████████████████████████████████████████████| 9868/9868 [05:07<00:00, 32.13it/s]


...

Train loss: 0.0084, train acc : 0.9896. Val loss: 0.1681, val acc : 0.8809.


100%|██████████████████████████████████████████████████████████████████████████████| 9868/9868 [04:55<00:00, 33.40it/s]


Train loss: 0.0086, train acc : 0.9893. Val loss: 0.1570, val acc : 0.8740.


 13%|█████████▋                                                                   | 3732/29603 [03:43<25:51, 16.68it/s]


KeyboardInterrupt: 

In [41]:
torch.save(model.state_dict(), r'C:\temp\big_model.h5')

In [ ]:
def make_prediction(dataloader, model):
    model.eval()
    predicted = []

    for images_batch in tqdm(dataloader, leave=True, desc="Images"):
        with torch.inference_mode():
            images_batch = images_batch.cuda()
            outputs = model(images_batch)
            probs = nn.functional.softmax(outputs, dim=-1).cpu().numpy()
            pred = np.argmax(probs, axis=1)
            predicted.extend(pred)
    
    return predicted

In [ ]:
def make_prediction(dataloader, model):
    model.eval()
    
    predictions = []
    fact = []

    with torch.inference_mode():
        index = 0
        for batch in tqdm(dataloader):
            inputs, labels = batch
            #labels = labels.long()
            inputs, labels = inputs.cuda(), labels.reshape(-1).cpu().numpy()

            outputs = model(inputs)
            probs = nn.functional.softmax(outputs, dim=-1).cpu().numpy()
            pred = np.argmax(probs, axis=1)
            predictions.append(pred)
            fact.append(labels)
            index += 1
            if index == 20:
                break
    return predictions, fact

In [906]:
predictions, fact = make_prediction(test_dataloader, model)

  0%|▏                                                                               | 19/9868 [00:00<07:25, 22.10it/s]


In [907]:
np.array(predictions).reshape(-1)

array([1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0], dtype=int64)

In [908]:
np.array(fact).reshape(-1)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0], dtype=int64)